In [5]:
import pandas as pd
import altair as alt

# Family Portrait

In [6]:
data=pd.read_csv('../public/lang-meta.tsv',sep='\t')
rel_data=pd.read_csv('./relationships.tsv',sep='\t')
subs = []
for idx in range(3):
    entity = "Rel " + str(idx + 1) + ": Entity"
    verb = "Rel " + str(idx + 1) + ": Verb"
    subs.append(rel_data.copy().rename(columns={entity: "Entity", verb: "Verb"})[['System', 'Entity', 'Verb']])
rel_data = pd.concat(subs).dropna()

In [7]:
keys = [
    'System', 
    'Carrier', 
    'Output Type',
    'Conceptual Model', 
    'Abstraction Mechanism', 
    'Source', 
    'Language Form',
    'Coded Domain', 
    'Execution Model', 
    'Alt API Available', 
    'Extensible',
    'Formal Definition Available', 
    'Language', 
    'Data manipulation',
    'Provides Accessibility', 
    'Juxtaposition strategy', 
    'Allowed Data Type',
    'Data model', 
    'Interaction source', 
    'Open Source', 
    'Dependent',
    'Mark Types', 
    'Series Types', 
    # 'Output Type Coded',
    # 'Embedded language', 
    'Coordinate Systems',
    "Annotation Support",
    "Model Formality",
    "Abstraction level"
    ]


binary_columns = [
    'Abstraction Mechanism',  
    'Alt API Available', 
    'Extensible',
    'Formal Definition Available', 
    'Provides Accessibility', 
    'Open Source', 
    'Dependent'
]

dumb_cols = [
    'Carrier', 
    'Output Type',
    'Conceptual Model',
    'Source', 
    'Language Form',
    'Coded Domain', 
    'Execution Model',
    'Language', 
    'Data manipulation',
    'Juxtaposition strategy', 
    'Allowed Data Type',
    'Data model', 
    'Interaction source', 
    # 'Output Type Coded',
    # 'Embedded language', 
    "Annotation Support",
    "Model Formality",
    "Abstraction level"
]

split_cols = [
    'Mark Types', 
    # 'Series Types', 
    'Coordinate Systems'
]

df = data[keys].copy()

gen_split_cols = []
for col in split_cols:
    col_types = set(",".join(df[col].dropna().tolist()).split(','))
    for col_type in col_types:
        new_col = str(col + col_type + 'new')
        gen_split_cols.append(new_col)
        df[new_col] = df[col].apply(lambda x : 1 if col_type in str(x) and str(x) != 'nan' else 0)


for bin_col in binary_columns: 
    df[bin_col] = df[bin_col].apply(lambda x : 0 if "no" in x.lower() else 0)
    

keep_cols = ['System'] + dumb_cols + binary_columns + gen_split_cols
out_df = pd.get_dummies(df[keep_cols], columns=dumb_cols)
ana_df = out_df.copy().drop(['System'], axis=1)

In [8]:
recs = [x for x in rel_data.to_dict('records') if x['Verb'] != 'Inspired by']

nodes = list(set([x['Entity'] for x in recs]))  + list(set([x['System'] for x in recs]))

child_nodes = {}
parent_nodes = {}
for node in nodes:
    child_nodes[node] = []
    parent_nodes[node] = []
for row in recs:
    child_nodes[row["Entity"]].append(row['System'])
    parent_nodes[row["System"]].append(row['Entity'])
# links
output = []
# initialize as nodes working set
working_set = [node for node in nodes if not len(parent_nodes[node])]
while len(working_set):
    childset = []
    for node in working_set:
        for child in child_nodes[node]:
            childset.append(child)
    outset = []
    for child in childset:
        for parent in parent_nodes[child]:
            if not parent in childset:
                outset.append(child)
    output.append(list(set(working_set)))
    working_set = list(set(outset))


def prepare_chart(output, verbs):
    pre_df_rows = []
    pos_map = {}
    system_order = {}
    for (idx, row) in enumerate(output):
        for system in row:
            system_order[system] = idx
    for row in output:
        for (jdx, system) in enumerate(row):
            order = system_order[system]
            frac = jdx / (len(row) - 1)
            pos_map[system] = (order, frac)
            pre_df_rows.append({"system": system, "order": order, "frac": frac})

    lines = []
    seen = {}
    for (idx, row) in enumerate(recs):
        src = row['Entity']
        trg = row['System']
        src_pos = pos_map[src]
        trg_pos = pos_map[trg]
        combo = (src, trg, src_pos, trg_pos)
        if combo in seen:
            continue
        seen[combo] = True
        lines.append({"detail": idx, "x": src_pos[1], "y": src_pos[0], "verb": row["Verb"], "system": src})
        lines.append({"detail": idx, "x": trg_pos[1], "y": trg_pos[0], "verb": row["Verb"], "system": trg})

    charts = []
    lines_df = pd.DataFrame(lines)
    for verb in verbs:
        base = alt.Chart(lines_df[lines_df['verb'] == verb])
        line = base.mark_line(interpolate="natural").encode(
            x=alt.X("x", axis=None), 
            y=alt.Y("y", scale=alt.Scale(reverse=True), axis=None), 
            detail="detail", 
            strokeDash="verb",
            color=alt.Color("verb", scale=alt.Scale(range=["#267F99", "#D16969", "#000000"])))
        text = base.mark_text(dy=-10).encode(x="x", y="y", text="system")
        point = base.mark_circle().encode(x="x", y="y")
        charts.append(line + point + text)
    return alt.layer(*charts).configure_axis(grid=False).configure_view(strokeWidth=0)

In [9]:
# Note that this chart is modified post hoc using figma (for label adjustment, fonts, etc)
output = [
    ['', 'Vega', '', 'P4'], 
    ['Gemini 1','Atom', '','', 'Vega-Lite', "PapARVis",  'P5', ''], 
    ['Gemini 2','Ivy',  'CompassQL', 'Scholz 3D Vis Language', 'VizGrammar', 'Multiclass-Density-Maps', 'Genome Spy', 'Gosling',  'SVL', 'VRIA', "Cicero", 'P6']]
prepare_chart(output, ['Compiles to', 'Wraps', 'Extends']).properties(height=155)

alt.LayerChart(...)